In [22]:
import os

import matplotlib.pyplot as plt
import pandas as pd

In [23]:
figure_path = 'figures_join_exit/'
os.makedirs(figure_path, exist_ok=True)

In [24]:
def filter_time_window(df, start_time_limit_seconds=0, stop_time_before_end_seconds=90):
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df['relative_time'] = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds()
    max_time = df['relative_time'].max()
    return df[
        (df['relative_time'] >= start_time_limit_seconds) &
        (df['relative_time'] <= max_time - stop_time_before_end_seconds)
    ]

In [25]:
def plot_metrics_by_node(df, field_name, figure_path='figures_join_exit/', start_time_limit_seconds=0, stop_time_before_end_seconds=90):
    os.makedirs(figure_path, exist_ok=True)
    filtered_df = df[df['field'] == field_name].copy()
    if filtered_df.empty:
        return

    if 'node' not in filtered_df.columns:
        return

    filtered_df['node'] = filtered_df['node'].str.split('_').str[-1].astype(int)
    filtered_df = filter_time_window(filtered_df, start_time_limit_seconds, stop_time_before_end_seconds)

    pivot_df = filtered_df.pivot(index='relative_time', columns='node', values='value')
    pivot_df = pivot_df.sort_index()
    pivot_df = pivot_df.ffill()
    pivot_df = pivot_df.bfill()

    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
    plt.rc('axes', axisbelow=True)
    plt.rcParams.update({'font.size': 10, 'xtick.labelsize': 8, 'ytick.labelsize': 8})

    plt.figure(figsize=(12, 7))
    for node in pivot_df.columns:
        plt.plot(pivot_df.index, pivot_df[node], label=f"Node {node}", linewidth=1.5, linestyle='-')

    plt.title(f"{field_name.replace('_', ' ').title()}", fontsize=14)
    plt.xlabel("Time (seconds)", fontsize=12)
    plt.ylabel(f"{field_name.replace('_', ' ').title()}", fontsize=12)
    plt.legend(title="Nodes", fontsize=10, loc="best")
    plt.grid(visible=True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plot_path = os.path.join(figure_path, f"{field_name.replace(' ', '_').lower()}.png")
    plt.savefig(plot_path, dpi=300)
    plt.close()


In [26]:
def read_file(path):
    return pd.read_csv(path, comment='#')

In [27]:
def process_data(df):
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        df = df.sort_values(by='timestamp')
        df['time_passed_seconds'] = (df['timestamp'] - df['timestamp'].iloc[0]).dt.total_seconds()
        df['time_passed_minutes'] = df['time_passed_seconds'] / 60
    if 'value' in df.columns:
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
    return df


In [28]:
def main(file_path, metric_names):
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        return

    data = read_file(file_path)
    data = process_data(data)

    unique_fields = data['field'].unique()

    for metric in metric_names:
        if metric in unique_fields:
            plot_metrics_by_node(data, metric, figure_path)
        else:
            print(f"Metric not found in data: {metric}")

In [29]:
metrics_path = '../metrics/'
file_path = os.path.join(metrics_path, "exit_node.csv")
metric_names = [
    'total_messages_sent',
    'total_sent',
    "active_peers",
    "current_round",
    "round_time",
]
main(file_path, metric_names)


Metric not found in data: total_messages_sent
